## Python装饰器的通俗理解
https://blog.csdn.net/buster_zr/article/details/81104551
http://python.jobbole.com/81683/


## 12步轻松搞定python装饰器
https://www.jianshu.com/p/d68c6da1587a

## 给妹子讲python-S01E22神奇的装饰器
https://zhuanlan.zhihu.com/p/33113561



## 这是我见过最全面的Python装饰器详解！没有学不会这种说法！
https://blog.csdn.net/qq_42156420/article/details/81169554


## 详解Python的装饰器
https://betacat.online/posts/2016-10-27/python-decorator/

### 为什么需要装饰器
假设你的程序实现了say_hello()和say_goodbye()两个函数。

In [14]:
def say_hello():
    print("hello!")
    
def say_goodbye():
    print("hello!") # bug here
    
if __name__ == '__main__':
    say_hello()
    say_goodbye()

hello!
hello!


但是在实际调用中，我们发现程序出错了，上面的代码打印了两个hello。经过调试你发现是say_goodbye()出错了。老板要求调用每个方法前都要记录进入函数的名称，比如这样：

```
[DEBUG]: Enter say_hello()
Hello!
[DEBUG]: Enter say_goodbye()
Goodbye!
```
小A是个毕业生，他是这样实现的。

In [24]:
def say_hello():
    print( "[DEBUG]: enter say_hello()")
    print("hello!")
    
def say_goodbye():
    print( "[DEBUG]: enter say_goodbye()")
    print("hello!") # bug here
    
if __name__ == '__main__':
    say_hello()
    say_goodbye()

[DEBUG]: enter say_hello()
hello!
[DEBUG]: enter say_goodbye()
hello!


很low吧？ 嗯是的。小B工作有一段时间了，他告诉小A可以这样写。

In [22]:
def debug():
    import inspect
    caller_name = inspect.stack()[1][3]
    print( "[DEBUG]: enter {}()".format(caller_name))

def say_hello():
    debug()
    print("hello!")
    
def say_goodbye():
    debug()
    print("hello!") # bug here
    
if __name__ == '__main__':
    say_hello()
    say_goodbye()

[DEBUG]: enter say_hello()
hello!
[DEBUG]: enter say_goodbye()
hello!


是不是好一点？那当然，但是每个业务函数里都要调用一下debug()函数，是不是很难受？万一老板说say相关的函数不用debug，do相关的才需要呢？

那么装饰器这时候应该登场了。

> 装饰器本质上是一个Python函数，它可以让其他函数在不需要做任何代码变动的前提下增加额外功能，装饰器的返回值也是一个函数对象。它经常用于有切面需求的场景，比如：插入日志、性能测试、事务处理、缓存、权限校验等场景。装饰器是解决这类问题的绝佳设计，有了装饰器，我们就可以抽离出大量与函数功能本身无关的雷同代码并继续重用。

概括的讲，**装饰器的作用就是为已经存在的函数或对象添加额外的功能。**

### 怎么写一个装饰器

在早些时候 (Python Version < 2.4，2004年以前)，为一个函数添加额外功能的写法是这样的。

In [27]:
def debug(func):
    def wrapper():
        print("[DEBUG]: enter {}()".format(func.__name__))
        return func()
    return wrapper

def say_hello():
    print("hello!")
    
def say_goodbye():
    print("hello!") # bug here

say_hello = debug(say_hello)  # 添加功能并保持原函数名不变
say_hello()

say_goodbye = debug(say_goodbye)  
say_goodbye()

[DEBUG]: enter say_hello()
hello!
[DEBUG]: enter say_goodbye()
hello!


#### @语法糖

上面的debug函数其实已经是一个装饰器了，它对原函数做了包装并返回了另外一个函数，额外添加了一些功能。因为这样写实在不太优雅，在后面版本的Python中支持**@语法糖**，下面代码等同于早期的写法。

In [28]:
def debug(func):
    def wrapper():
        print("[DEBUG]: enter {}()".format(func.__name__))
        return func()
    return wrapper

@debug
def say_hello():
    print("hello!")

@debug
def say_goodbye():
    print("hello!")

say_hello()

[DEBUG]: enter say_hello()
hello!


`@debug` 等同于 `say_hello = debug(say_hello) `

直接调用函数 debug(say_hello) 

**Python何时执行装饰器**

装饰器的一个关键特性是，它们在被装饰的函数定义之后立即运行。这通常是在导入时（即 Python 加载模块时）。

这是最简单的装饰器，但是有一个问题，如果被装饰的函数需要传入参数，那么这个装饰器就坏了。

In [33]:
def debug(func):
    def wrapper():
        print("[DEBUG]: enter {}()".format(func.__name__))
        return func()
    return wrapper

@debug
def say(something):
    print('hello，{}！'.format(something))
    
say('love')

TypeError: wrapper() takes 0 positional arguments but 1 was given

#### 带参数

因为返回的函数并不能接受参数，你可以指定装饰器函数wrapper接受和原函数一样的参数，比如：

In [32]:
def debug(func):
    def wrapper(something):
        print("[DEBUG]: enter {}()".format(func.__name__))
        return func(something)
    return wrapper

@debug
def say(something):
    print('hello，{}！'.format(something))
    
say('love')

[DEBUG]: enter say()
hello，love！


这样你就解决了一个问题，但又多了N个问题。因为函数有千千万，你只管你自己的函数，别人的函数参数是什么样子，鬼知道？还好Python提供了可变参数*args和关键字参数**kwargs，有了这两个参数，装饰器就可以用于任意目标函数了。

In [37]:
def debug(func):
    def wrapper(*args,**kwargs):
        print("[DEBUG]: enter {}()".format(func.__name__))
        return func(*args,**kwargs)
    return wrapper

@debug
def say(something):
    print('hello，{}！'.format(something))
    
@debug
def say_hello():
    print("hello!")

@debug
def say_goodbye(method='kissing'):
    print("Say goodbye with {}".format(method))
    
say('mylove')
say_hello()
say_goodbye(method='kissing')

[DEBUG]: enter say()
hello，mylove！
[DEBUG]: enter say_hello()
hello!
[DEBUG]: enter say_goodbye()
Say goodbye with kissing


上诉是两层嵌套的decorator。至此，你已完全掌握初级的装饰器写法。

### 高级一点的装饰器

#### 带参数的装饰器

如果装饰器本身需要传入参数

我们前文的装饰器需要完成的功能不仅仅是能在进入某个函数后打出log信息，而且还需指定log的级别，那么装饰器就会是这样的。

In [91]:
def logging(level):
    def wrapper(func):
        def inner_wrapper(*args,**kwargs):
            print('[{level}]: enter function {func}'.format(level=level,func=func.__name__))
            return func(*args,**kwargs)
        return inner_wrapper
    return wrapper

@logging(level = "INFO")
def say(something):
    print('hello，{}！'.format(something))
    
if __name__ == '__main__':
    say('mylove')

[INFO]: enter function say
hello，mylove！


和两层嵌套的decorator相比，3层嵌套的效果是这样的：

`say = logging(level='INFO')(say)`

我们来剖析上面的语句，首先执行`logging(level='INFO')`，返回的是wrapper函数，`wrapper(say)`再调用返回的函数，参数是say函数，返回值最终是inner_wrapper 函数。

是不是有一些晕？你可以这么理解，当带参数的装饰器被打在某个函数上时，比如@logging(level='DEBUG')，它其实是一个函数，会马上被执行，只要这个它返回的结果是一个装饰器时，那就没问题。细细再体会一下。

#### 基于类实现的装饰器

装饰器函数其实是这样一个接口约束，它必须接受一个callable对象作为参数，然后返回一个callable对象。在Python中一般callable对象都是函数，但也有例外。只要某个对象重载了\__call__()方法，那么这个对象就是callable的。

In [61]:
class Test():
    def __call__(self):
        print('call me!')

t = Test()
t()

call me!


像\__call__这样前后都带下划线的方法在Python中被称为内置方法，有时候也被称为魔法方法。重载这些魔法方法一般会改变对象的内部行为。上面这个例子就让一个类对象拥有了被调用的行为。

##### \__call__

一个对象实例可以有自己的属性和方法，当我们调用实例方法时，我们用instance.method()来调用。能不能直接在实例本身上调用呢？在Python中，答案是肯定的。

任何类，只需要定义一个\__call__()方法，就可以直接对实例进行调用。请看示例：

In [62]:
class Student(object):
    def __init__(self, name):
        self.name = name

    def __call__(self):
        print('My name is %s.' % self.name)
        
s = Student('Jamie')
s()

My name is Jamie.


\__call__()还可以定义参数。对实例进行直接调用就好比对一个函数进行调用一样，所以你完全可以把对象看成函数，把函数看成对象，因为这两者之间本来就没啥根本的区别。

如果你把对象看成函数，那么函数本身其实也可以在运行期动态创建出来，因为类的实例都是运行期创建出来的，这么一来，我们就模糊了对象和函数的界限。

那么，怎么判断一个变量是对象还是函数呢？其实，更多的时候，我们需要判断一个对象是否能被调用，能被调用的对象就是一个Callable对象，比如函数和我们上面定义的带有\__call__()的类实例.

通过callable()函数，我们就可以判断一个对象是否是“可调用”对象。

回到装饰器上的概念上来，装饰器要求接受一个callable对象，并返回一个callable对象（不太严谨，详见后文）。那么用类来实现也是也可以的。我们可以让类的构造函数\__init__()接受一个函数，然后重载\__call__()并返回一个函数，也可以达到装饰器函数的效果。

In [53]:
class logging(object):
    def __init__(self,func):
        self.func = func
        
    def __call__(self,*args,**kwargs):
        print("[DEBUG]: enter function {}()".format(self.func.__name__))
        return self.func(*args,**kwargs)
    
@logging
def say(something):
    print('hello，{}！'.format(something))
    
if __name__ == '__main__':
    say('mylove')

[DEBUG]: enter function say()
hello，mylove！


等同于

In [59]:
class logging(object):
    def __init__(self,func):
        self.func = func
        
    def __call__(self,*args,**kwargs):
        print("[DEBUG]: enter function {}()".format(self.func.__name__))
        return self.func(*args,**kwargs)

def say(something):
    print('hello，{}！'.format(something))
    
if __name__ == '__main__':
    say = logging(say)
    say('mylove')

[DEBUG]: enter function say()
hello，mylove！


#### 带参数的类装饰器

如果需要通过类形式实现带参数的装饰器，那么会比前面的例子稍微复杂一点。那么在构造函数里接受的就不是一个函数，而是传入的参数。通过类把这些参数保存起来。然后在重载\__call__方法是就需要接受一个函数并返回一个函数。

In [68]:
class logging(object):
    def __init__(self,level):  
        self.level = level
        
    def __call__(self,func):
        def wrapper(*args,**kwargs):
            print("[{level}]: enter function {func}()".format(level=self.level,func=func.__name__))
            return func(*args,**kwargs)
        return wrapper
    
@logging(level='INFO') # 创建实例 有了__init__方法，在创建实例的时候，就不能传入空的参数了，必须传入与__init__方法匹配的参数，但self不需要传
def say(something):
    print('hello，{}！'.format(something))
    
if __name__ == '__main__':
    say('mylove') # 实例调用，必须传入与__call__方法匹配的参数，但self不需要传

[INFO]: enter function say()
hello，mylove！


等同于

In [82]:
class logging(object):
    def __init__(self,level):
        self.level = level
        
    def __call__(self,func):
        def wrapper(*args,**kwargs):
            print("[{level}]: enter function {func}()".format(level=self.level,func=func.__name__))
            return func(*args,**kwargs)
        return wrapper
    
def say(something):
    print('hello，{}！'.format(something))
    
if __name__ == '__main__':
    say = logging(level='INFO')(say)
    say('mylove')

[INFO]: enter function say()
hello，mylove！


### 错误的函数签名和文档

装饰器装饰过的函数看上去名字没变，其实已经变了。

In [104]:
def logging(func):
    def wrapper(*args,**kwargs):
        """print log before a function."""
        print('[DEBUG]: enter function {}()'.format(func.__name__))
        return func(*args,**kwargs)
    return wrapper

@logging
def say(something):
    """say something"""
    print('hello，{}！'.format(something))
    
if __name__ == '__main__':
    say('mylove')
    print(say.__name__)
    print(say.__doc__)

[DEBUG]: enter function say()
hello，mylove！
wrapper
print log before a function.


>**\__doc__**

>函式体的第一个语句可以是三引号括起来的字符串， 这个字符串就是函数的文档字符串，或称为docstring。我们可以使用print(function.\__doc__)输出文档.文档字符串主要用于描述一些关于函数的信息，让用户交互地浏览和输出。建议养成在代码中添加文档字符串的好习惯。

为什么会这样呢？只要你想想装饰器的语法糖@代替的东西就明白了。@等同于这样的写法。

`say = logging(say)`

logging其实返回的函数名字刚好是wrapper，那么上面的这个语句刚好就是把这个结果赋值给say，say的__name__自然也就是wrapper了，不仅仅是name，其他属性也都是来自wrapper，比如doc，source等等。

使用标准库里的functools.wraps，可以基本解决这个问题。


In [102]:
from functools import wraps

def logging(func):
    @wraps(func)
    def wrapper(*args,**kwargs):
        """print log before a function."""
        print('[DEBUG]: enter function {}()'.format(func.__name__))
        return func(*args,**kwargs)
    return wrapper

@logging
def say(something):
    """say something"""
    print('hello，{}！'.format(something))
    
if __name__ == '__main__':
    say('mylove')
    print(say.__name__)
    print(say.__doc__)

[DEBUG]: enter function say()
hello，mylove！
say
say something


看上去不错！主要问题解决了，但其实还不太完美。因为函数的签名和源码还是拿不到的。